In [1]:
import cv2
import sys
from random import randint
import time


## Create a method to use to create the tracker

In [2]:
trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']

def createTrackerByName(trackerType):
  # Create a tracker based on tracker name
  if trackerType.upper() == trackerTypes[0]:
    tracker = cv2.TrackerBoosting_create()
  elif trackerType.upper() == trackerTypes[1]: 
    tracker = cv2.TrackerMIL_create()
  elif trackerType.upper() == trackerTypes[2]:
    tracker = cv2.TrackerKCF_create()
  elif trackerType.upper() == trackerTypes[3]:
    tracker = cv2.TrackerTLD_create()
  elif trackerType.upper() == trackerTypes[4]:
    tracker = cv2.TrackerMedianFlow_create()
  elif trackerType.upper() == trackerTypes[5]:
    print("Ensure you have the goturn caffemodel in your directory. To download it, see reference 4")
    tracker = cv2.TrackerGOTURN_create()
  elif trackerType.upper() == trackerTypes[6]:
    tracker = cv2.TrackerMOSSE_create()
  elif trackerType.upper() == trackerTypes[7]:
    tracker = cv2.TrackerCSRT_create()
  else:
    tracker = None
    print('Incorrect tracker name')
    print('Available trackers are:')
    for t in trackerTypes:
      print(t)
     
  return tracker

 ## Attempt to read the video

In [3]:
#Set video0 to a video with ~4 minutes of content.
#Set video1 to a car chase
video0 = "ball rebounding and bouncing all around the screen with trails and music 1080p HD Animation.mp4"
video1 = "1-Stolen Dodge Hellcat Outruns Chopper in Houston Police Chase! Driver Almost Makes it.mp4"
video = video1
capture = cv2.VideoCapture(video)

#Attempt to read a frame
success, frame = capture.read()
if not success:
    print("Video doesn't exist.")
    sys.exit(1)

## Draw bounding boxes around the object to be tracked

In [4]:
boundBoxes = []
# color is going to be initialized to a random color for every object
colors = []


In [5]:


# A loop to run the object selection till "q" is pressed
while not True:
    boundbox = cv2.selectROI('tracker', frame)
    boundBoxes.append(boundbox)
    colors.append((randint(0,255), randint(0, 255), randint(0, 255)))
    print("Press q/ESC to quit selecting boxes and start tracking")
    print("Press Space to set the next object to select")
    k = cv2.waitKey(0)
    if (k == 113 or k==27):
        break        
        
cv2.destroyWindow("tracker")
print("Selected bounding boxes{}".format(boundBoxes))

Selected bounding boxes[]


## Set bounding box and color to a fixed location so test is reproducible

In [6]:
## Define the bounding box for the videos so you can reproduce the video
if video==video0:
    boundBoxes = [(0, 0, 11, 16)]
    colors = [(44, 156, 222)]
if video ==video1:
    boundBoxes = [(258, 156, 76, 32)]
    colors = [(44, 156, 222)]

## Initialize a Multi-tracker to track the boundingBoxes

In [7]:
#Select a tracker from the trackerTypes
trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
#Enter the name of the tracker or index it 
trackerType = "GOTURN"

#Create the multitracker object
multiTracker = cv2.MultiTracker_create()

#initialize the multitracker
for boundbox in boundBoxes:
    multiTracker.add(createTrackerByName(trackerType), frame, boundbox)

Ensure you have the goturn caffemodel in your directory. To download it, see reference 4


## Define the VideoWriter

In [8]:
#Get the frame dimensions
frame_width = int(capture.get(3))
frame_height = int(capture.get(4))

out = cv2.VideoWriter(trackerType+".avi",cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
## text that shows the frames per second
originalFPS = round(capture.get(cv2.CAP_PROP_FPS), 1)

## Process the video and begin object tracking

In [9]:
def run_tracking(capture, multiTracker, trackerType):
    start = time.time()
    frameCount = 1
    #Process video and track objects. I use 1695 to make the algorithms compare the same frames in the video.
    while capture.isOpened() and frameCount < 1695:
        frameCount+=1
        runningTime = time.time()
        success, frame = capture.read()
        if not success:
            break
        #get updated location of objects being tracked in subsequent fram
        success, boxes = multiTracker.update(frame)
        elapsedSeconds = time.time() - start
        #Draw box around tracked objects
        p1  = []
        p2 = []
        for i, newbox in enumerate(boxes):
            p1 = (int(newbox[0]), int(newbox[1]))
            p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
            

        #Show the frame and insert appropriate text into the frame
        cv2.putText(frame, trackerType, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA )
        if success:
            avgTrackingFPS = "Average Tracking FPS: " + str(round(frameCount/elapsedSeconds+0.000000001, 3))
            runTrackingFPS = "Current Tracking FPS: " + str(round(1/(time.time()-runningTime+0.000000001), 3))
            oFPS = "Input FPS: " + str(originalFPS)
            cv2.putText(frame, avgTrackingFPS, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA )
            cv2.putText(frame, runTrackingFPS, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA )
            cv2.putText(frame, oFPS, (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA )
            cv2.rectangle(frame, p1, p2, colors[i], 2, 1)
        else:
            failText = "Tracking Failure"
            cv2.putText(frame, failText, (180, 240), cv2.FONT_HERSHEY_COMPLEX, 1.0, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.imshow(trackerType, frame)
        out.write(frame)
        #Quit if the Escape button is pressed
        if cv2.waitKey(1) & 0xFF ==27:
            out.release()
            break
    cv2.destroyAllWindows()
    out.release()
    print(frameCount)

In [10]:
%time run_tracking(capture, multiTracker, trackerType)

error: OpenCV(4.0.0) C:\projects\opencv-python\opencv\modules\core\src\alloc.cpp:55: error: (-4:Insufficient memory) Failed to allocate 36422871600 bytes in function 'cv::OutOfMemoryError'


In [11]:
print(success)
cv2.destroyAllWindows()

True


## References
1. https://www.learnopencv.com/multitracker-multiple-object-tracking-using-opencv-c-python/
2. Opencv docs
3. https://www.learnopencv.com/goturn-deep-learning-based-object-tracking/
4. https://www.dropbox.com/sh/77frbrkmf9ojfm6/AACgY7-wSfj-LIyYcOgUSZ0Ua?dl=0